# Randomforest NLP

In [49]:
import numpy as np
import pandas as pd

In [50]:
import warnings
warnings.filterwarnings("ignore")

In [51]:
X_train = pd.read_csv('https://raw.githubusercontent.com/wongwara/Jobseeker_Baymax/main/Final/data%20splitted/X_train.csv', index_col=[0])
X_test = pd.read_csv('https://raw.githubusercontent.com/wongwara/Jobseeker_Baymax/main/Final/data%20splitted/X_test.csv', index_col=[0])
y_train = pd.read_csv('https://raw.githubusercontent.com/wongwara/Jobseeker_Baymax/main/Final/data%20splitted/y_train.csv', index_col=[0])
y_test = pd.read_csv('https://raw.githubusercontent.com/wongwara/Jobseeker_Baymax/main/Final/data%20splitted/y_test.csv', index_col=[0])
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [52]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(510, 7760)
(510,)
(128, 7760)
(128,)


# Scale the data

In [53]:
#Import MinMaxScaler from Sklearn
#from sklearn.preprocessing import MinMaxScaler

#Instantiate a MinMaxScaler and save it into a variable called scaler
#scaler = MinMaxScaler()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#Fit the scaler on the training set
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
X_train

array([[-0.72618462, -0.67761149,  0.        , ..., -0.08889762,
        -0.04432422, -0.04432422],
       [-0.72618462, -0.67761149,  0.        , ..., -0.08889762,
        -0.04432422, -0.04432422],
       [ 0.10982928,  0.17991174,  0.        , ..., -0.08889762,
        -0.04432422, -0.04432422],
       ...,
       [ 0.52783623, -0.67761149,  0.        , ..., -0.08889762,
        -0.04432422, -0.04432422],
       [-0.72618462, -1.10637311,  0.        , ..., -0.08889762,
        -0.04432422, -0.04432422],
       [-0.72618462, -0.67761149,  0.        , ..., -0.08889762,
        -0.04432422, -0.04432422]])

In [55]:
# K-fold split
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
cv = StratifiedShuffleSplit(n_splits=5, test_size=.20, random_state=42)

# Assess Baseline Model

In [56]:
y_mean = y_train.mean()
y_mean

4.433333333333334

In [57]:
#y_mode = y_train.mode()
#y_mode

In [58]:
y_shape = (len(X_train), 1)
y_base = np.full(y_train.shape, y_mean)

In [59]:
#from sklearn.metrics import accuracy_score, f1_score
#print(accuracy_score(y_train, y_base))
#print(f1_score(y_train, y_base, average='weighted'))

In [60]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

print(mse(y_train, y_base, squared=False))
print(mae(y_train, y_base))

2.8419936848232377
2.469411764705882


# Train Initial RandomForest with Default Hyperparameter

In [61]:
# Import the RandomForestClassifier from sklearn.ensemble
from sklearn.ensemble import RandomForestClassifier

# Instantiate the RandomForestClassifier class called rf1 with a random state=8
rf1 = RandomForestClassifier(random_state=8)

# Fit the RandomForest model
rf1.fit(X_train, y_train)

# Calculate the accuracy of the RandomForest on the training set and print them
print(rf1.score(X_train, y_train))

0.9921568627450981


In [62]:
#Calculate the RMSE score of the model on the training and validation sets and print
y_train_rf1_preds = rf1.predict(X_train)
# y_val_rf1_preds = rf1.predict(X_val)

print("MSE y_train:",mse(y_train, y_train_rf1_preds, squared=False))
# print("MSE y_val:",mse(y_val, y_val_rf1_preds, squared=False))
print("MAE y_train:",mae(y_train, y_train_rf1_preds))
# print("MAE y_val",mae(y_val, y_val_rf1_preds))

MSE y_train: 0.2581988897471611
MAE y_train: 0.0196078431372549


# RandomForeat with GridsearchCV

In [63]:
# Import the necessary libraries and modules
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [64]:
# Instantiate the random forest classifier and define the parameter grid for the grid search
rfc = RandomForestClassifier()
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [65]:
# Create a GridSearchCV object and fit the training data
grid_search = GridSearchCV(rfc, param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

In [66]:
# Print the best parameters and the best score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best score:  0.48235294117647054


In [67]:
# Use the best estimator to make predictions on the test data
rfc_best = grid_search.best_estimator_
y_pred_rfc = rfc_best.predict(X_test)

In [77]:
from sklearn.metrics import accuracy_score, f1_score

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred_rfc)
print("Accuracy score:", accuracy)

# Calculate F1 score --> Target is multiclass but average='binary'
#f1 = f1_score(y_test, y_pred)
#print("F1 score:", f1)

Accuracy score: 0.5


# RandomForest with RandomizedSearchCV

In [69]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [70]:
# Define the parameter grid for the RandomizedSearchCV
param_dist = {'n_estimators': randint(100, 1000),
              'max_features': ['auto', 'sqrt', 'log2'],
              'max_depth': [10, 20, 30, None],
              'min_samples_split': randint(2, 20),
              'min_samples_leaf': randint(1, 20),
              'bootstrap': [True, False]}

In [71]:
# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(rfc, param_distributions=param_dist,
                                   n_iter=10, cv=5, scoring='accuracy', random_state=42)

In [72]:
# Fit the RandomizedSearchCV object to the training data
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, None],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff5e2ad3f70>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff5e2b254c0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff5e19587c0>},
                   random_state=42, scoring='accuracy')

In [73]:
# Get the best model
rsf_model = random_search.best_estimator_

In [74]:
# Make predictions on the test set
y_pred_rfs = best_model.predict(X_test)

In [78]:
# Compute the accuracy and F1 scores
accuracy = accuracy_score(y_test, y_pred_rfs)
# f1 = f1_score(y_test, y_pred)

In [79]:
# Print the results
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)
print("Accuracy score: ", accuracy)
# print("F1 score: ", f1)

Best parameters:  {'bootstrap': False, 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 918}
Best score:  0.4901960784313726
Accuracy score:  0.484375


# Result

For Gridsearch, best parameter is 'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100.
- Best score is 0.48235294117647054
- Accuracy score is 0.5

For Randomsearch, best parameter is 'bootstrap': False, 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 918.
- Best score is 0.4901960784313726
- Test accuracy score is 0.484375

## __Conclusion__
Best model is __RandomForest with GridSearchCV__.